In [1]:
# * Edit alternate-color (the partitions are right)
# * Assimiation so that the variants are corrent (regenerate with less variants)
# * Copy of the baseline model
# * Generate 1 matplotlib image

# ??? 30-40 mintues
# (100%) -> 96 (generation) (3) inference baseline run and 
# 
# Top candidate (- 2 )
# candiadte by the baseline

from glob import glob
import os
%pylab inline
import matplotlib.pyplot as plt
from copy import deepcopy

rx_dict = {
    'candidate_num' : re.compile(r"Candidate: c(?P<candidate_num>\d)\n"),
    'concept' : re.compile(r"Concept: (?P<concept>.*)\n"),
    'candidate' : re.compile(r"Candidate Name: (?P<candidate>.*)\n"),
    'end' : re.compile(r"No discriminator found for puzzle (?P<cpath>.*)\n"),
    'bscores' : re.compile(r"The scores were (?P<score_list>\[[\d., e-]+\]).\n"),
    'bcandidate' : re.compile(r"Best concept is @ idx (\d) ie: (?P<bcpath>.*)\n")
}


def _parse_line(line):
    for key, rx in rx_dict.items():
        match = rx.search(line)
        if match:
            return (key, match)
    return (None, None)

def _parse_file(file_path):
    data = dict()
    data['name'] = str()
    data['concepts'] = list()
    data['baseline'] = dict()

    with open(file_path, 'r') as fp:
        line = fp.readline()

        concept = dict()
        while ('No discriminator found' not in line) and ('The scores were' not in line):                
            key, match = _parse_line(line)
            concept[key] = list(match.groups())[0]
            line = fp.readline()
            if line == "\n":
                line = fp.readline()
                data['concepts'].append(deepcopy(concept))
                concept = dict()
        
        if 'The scores were' not in line:        
            key, match = _parse_line(line)
            data['name'] = match.groups()[0]
            line = fp.readline()
        while line:
            key, match = _parse_line(line)
            if 'Best concept' in line:
                data['baseline'][key] = list(match.groups())[1]
                data['baseline']['bcandidate_num'] = list(match.groups())[0]
                
            line = fp.readline()
        
    return data
    


Populating the interactive namespace from numpy and matplotlib


In [64]:

def viz(puzzle, output_path):
    try:
        import matplotlib
        import matplotlib.pyplot as plt
        import matplotlib.image as mpimg
        import matplotlib.patches as patches
        import matplotlib.gridspec as gridspec

        output = (f"/home/ubuntu/vdp-tool-chain-repo/data/clevr-variants/{puzzle}.out")
        images = sorted(glob(f"/home/ubuntu/vdp-tool-chain-repo/data/output/images/{puzzle}/*"), key= lambda x: os.path.basename(x))
        output = _parse_file(output)

        name = os.path.basename(output['name'])[:-1]
        concept0 = output['concepts'][0]['concept']
        candidate0 = os.path.basename(output['concepts'][0]['candidate']).split(".")[0]
        bcandidate = output['baseline']['bcandidate_num']
        concept0 = re.sub("(.{64})", r"\1\n\t", concept0, 0, re.DOTALL)

        fig = plt.figure(constrained_layout=False, figsize=(int(12.5*1.5), int(12.5*1.5)))
        # gs1 = fig.add_gridspec(nrows=2, ncols=2, left=0.05, right=0.4, bottom=0.35,
                                # wspace=0.05)
        gs1 = fig.add_gridspec(nrows=2, ncols=2, left=0.45, right=0.98, bottom=0.35,
                                wspace=0.05)
                                
        # gs2 = fig.add_gridspec(nrows=3, ncols=1, left=0.2, right=0.98,
        #                         hspace=0.25)
        gs2 = fig.add_gridspec(nrows=3, ncols=1, left=0.02, right=0.40,
                                hspace=0.25, wspace=0.5)

        gs3 = fig.add_gridspec(nrows=1, ncols=1, left=0.45, right=0.98, top=0.3,
                                hspace=0.25)


        ax0 = fig.add_subplot(gs3[:, :])
        ax0.title.set_text('Results')

        ax1 = fig.add_subplot(gs1[:-1, :])
        ax1.title.set_text('Candidate Image 1 (Answer)')
        ax1.imshow(mpimg.imread(images[0]))

        ax2 = fig.add_subplot(gs1[-1, :-1])
        ax2.title.set_text('Candidate Image 2')
        ax2.imshow(mpimg.imread(images[1]))

        ax3 = fig.add_subplot(gs1[-1, -1])
        ax3.title.set_text('Candidate Image 3')
        ax3.imshow(mpimg.imread(images[2]))
        ax4 = fig.add_subplot(gs2[0, :])
        ax4.title.set_text('Train Image 1')
        ax4.imshow(mpimg.imread(images[3]))

        ax5 = fig.add_subplot(gs2[1, :])
        ax5.title.set_text('Train Image 2')
        ax5.imshow(mpimg.imread(images[4]))

        ax6 = fig.add_subplot(gs2[2, :])
        ax6.title.set_text('Train Image 3')
        ax6.imshow(mpimg.imread(images[5]))

        ax0.axis('off')
        ax1.axis('off')
        ax2.axis('off')
        ax3.axis('off')
        ax4.axis('off')
        ax5.axis('off')
        ax6.axis('off')

        txtargs = {"transform" : ax0.transAxes, "fontsize" : 12, "verticalalignment" : 'top', "horizontalalignment" : 'left'}

        # place a text box in upper left in axes coords
        name_text = f"""Puzzle Name: {name}
        Puzzle Description: There are two spheres in a line with the same color
        Solver Selected Candidate: Candidate {int(candidate0) + 1}
        Baseline Selected Candidate: Candidate {int(bcandidate) + 1}
        Solver identified concept:\n\t{concept0}""".expandtabs()



        ax0.text(0.05, 0.9, name_text, **txtargs)
        plt.savefig(output_path)
    except:
        print(puzzle)


In [63]:
# puzzle = "breaking-3"
# output_path

# for p in list(map(lambda x : os.path.basename(x).split[0], glob("clevr-variants/*-[0-9]?.out"))):
    # print(p)



TypeError: catching classes that do not inherit from BaseException is not allowed